# SnowEx Ground-Based Measurements and Data Packages

Outline:
* Central measurements to SnowEx campaigns? (standardized protocol)
* Campaign objectives by year
* Sampling designs and naming conventions for each campaign

## Set Up Computing Environment

In [ ]:
# import python packages
import ipyleaflet
from ipyleaflet import Map, GeoData, Rectangle, basemaps, LayersControl, basemap_to_tiles, TileLayer, SplitMapControl, Polygon

import ipywidgets
import datetime
import re
import geopandas as gpd

## Explore the Spatial Extent of Field Campaign Data

In [ ]:
bbox = [-125, 49, -102, 31]
west, north, east, south = bbox
bbox_ctr = [0.5*(north+south), 0.5*(west+east)]

In [ ]:
m = Map(basemap=basemaps.CartoDB.Positron, center=bbox_ctr, zoom=4)
rectangle = Rectangle(bounds=((south, west), (north, east))) #SW and NE corners of the rectangle (lat, lon)
m.add_layer(rectangle)
m

# more info on available basemaps here: https://ipyleaflet.readthedocs.io/en/latest/map_and_basemaps/basemaps.html

## Query the Database to add Spatial Data to Our Map

In [ ]:
# database imports
from snowexsql.db import get_db
from snowexsql.data import PointData, LayerData, ImageData, SiteData
from snowexsql.conversions import query_to_geopandas, query_to_pandas

In [ ]:
# load the database
db_name = 'snow:hackweek@db.snowexdata.org/snowex'
engine, session = get_db(db_name)

## Let's find out where we have liquid water content (LWC) data 

**Pointer -->**  LWC data is in the LayerData table, because data at a single location were measured as a profile on the pit wall face (i.e has a vertical dimension) 

In [ ]:
# query the LayerData for all LWC values
# qry = session.query(LayerData).filter(LayerData.type == 'lwc_vol')

# query the geometry property of PointData
# qry = session.query(LayerData.geom).distinct()

# query the LayerData for all LWC values
qry = session.query(LayerData.longitude, LayerData.latitude).filter(LayerData.type == 'lwc_vol').distinct() # 

# # query the geometry property of PointData
# qry = query(LayerData.geom).distinct()

# # limit the number of entries
# qry = qry.limit(2000)

# convert query to geopandas df
df = query_to_pandas(qry, engine)

df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
print(type(df))

# # convert query to geopandas df
# df = query_to_geopandas(qry, engine)

print(df.head())

# how many did we retrieve?
print(f'{len(df.index)} records returned!')

session.close()

In [ ]:
type(df)

## Let's add these points to our map!

In [ ]:
m = Map(basemap=basemaps.CartoDB.Positron, center=bbox_ctr, zoom=6)

# geo_data = GeoData(geo_dataframe = df)

geo_data = GeoData(geo_dataframe = df,
    style={'color': 'black', 'radius':8, 'fillColor': '#3366cc', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
    hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
    point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
    name = 'lwc obs.')

m.add_layer(geo_data) 
m.add_control(LayersControl())
m

## Main body here 

## Share Your Thoughts

1. Navigate to this **Google Slides** page
2. Add your thought on data applications now that you know what measurements we have and roughly where they occur

## Recap

## References